In [81]:
from sportsipy.nhl.teams import Teams
from scipy.stats import pearsonr
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

print('Correlation(WinRatio,GoalDifferential)\n')
      
# https://towardsdatascience.com/web-scraping-nba-stats-4b4f8c525994
def getGa(abbr,year):
    # for abbr in abbr_list:
    url = 'https://www.hockey-reference.com/teams/' + abbr + '/' + str(year) + '.html'
    html = urlopen(url)
    soup = BeautifulSoup(html,features="lxml")
    
    # avoid the first header row
    rows = soup.findAll('tr')[1:]
    all_stats = [[td.getText() for td in rows[i].findAll('td')]for i in range(len(rows))]
    all_stats2 = [e for e in all_stats if e]
    all_stats3=[x for x in all_stats2 if len(x)>3]
    team_stats = all_stats3[0]
    
    goals_against = team_stats[8]
    return(goals_against)

corrd = {}
for c,year in enumerate(range(1980, 2021)):
    teams = list(Teams(year))
    statd = {}
    for i,team in enumerate(teams[0:len(teams)]):
        G = team.games_played
        Rwin = round(team.wins/team.games_played,3)
        GF = team.goals_for
        GA = int(getGa(team.abbreviation,year))
        statd[i] = {'Team' : team.name,
                    'W' : team.wins,
                    'L' : team.losses,
                    'OTL' : team.overtime_losses,
                    'G' : G,
                    'Rwin' : round(((.5*team.overtime_losses)+team.wins)/G,3),
                    'GF' : GF,
                    'GA' : GA,
                    'GD' : GF-GA,
                    'GD_per' : (GF-GA)/G} 

    df = pd.DataFrame(statd)
    df = df.transpose()
    
    x = list(df['Rwin'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Iwin'] =  index
  
    x = list(df['GD'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Idiff'] =  index
    
    valuCorr, _ = pearsonr(df['GD'].tolist(), df['Rwin'].tolist())
    print('%s: %s' % (year,round(valuCorr,3)))

Correlation(WinRatio,GoalDifferential)

2019: 0.975
2020: 0.967


In [84]:
teams = list(Teams(2015))
statd = {}
    
for i,team in enumerate(teams[0:len(teams)]):
    GP = team.games_played
    Rwin = round(team.wins/team.games_played,4)
    GF = team.goals_for
    GA = int(getGa(team.abbreviation,year))
    statd[i] = {'Team' : team.name,
                'W' : team.wins,
                'L' : team.losses,
                'OTL' : team.overtime_losses,                
                'G' : G,
                'Rwin' : round(((.5*team.overtime_losses)+team.wins)/G,3),
                'GF' : GF,
                'GA' : GA,
                'GD' : GF-GA,
                'GD_per' : (GF-GA)/G} 
        
df = pd.DataFrame(statd)
df = df.transpose()

x = list(df['Rwin'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Iwin'] =  index

x = list(df['GD'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Idiff'] =  index

df = df[['Team','W','OTL','L','G','GF','GA','GD','Rwin','GD_per','Iwin','Idiff']]
df = df.sort_values('Rwin',ascending=False)
print('%s NHL Season - Ranked by [Iwin] -> Ratio(Games Won:Total Games) [Rwin] \n' % year)
print(df.to_string(index=False))

df = df[['Team','W','OTL','L','G','GF','GA','GD','Rwin','GD_per','Iwin','Idiff']]
df = df.sort_values('GD',ascending=False)
print('\n%s NHL Season - Ranked by [Idiff] -> Goal Differential [GD] \n' % year)
print(df.to_string(index=False))

2020 NHL Season - Ranked by [Iwin] -> Ratio(Games Won:Total Games) [Rwin] 

                  Team   W OTL   L   G   GF   GA   GD   Rwin     GD_per  Iwin  Idiff
      New York Rangers  53   7  22  71  248  220   28  0.796   0.394366     0      6
    Montreal Canadiens  50  10  22  71  214  220   -6  0.775  -0.084507     1     20
         Anaheim Ducks  51   7  24  71  228  225    3  0.768  0.0422535     2     18
       St. Louis Blues  51   7  24  71  239  190   49  0.768   0.690141     2      3
   Tampa Bay Lightning  50   8  24  71  259  194   65  0.761   0.915493     4      1
   Nashville Predators  47  10  25  71  226  214   12  0.732   0.169014     5     14
    Chicago Blackhawks  48   6  28  71  220  214    6  0.718   0.084507     6     17
    New York Islanders  47   7  28  71  245  190   55  0.711   0.774648     7      2
     Vancouver Canucks  48   5  29  71  236  214   22  0.711   0.309859     7      9
   Washington Capitals  45  11  26  71  237  212   25  0.711   0.352113   